In [5]:
import os
import numpy as np
import glob
import os.path as op
import nibabel as nib
from ipyparallel import Client

In [6]:
home_dir =  '/home/dnl/habitization/'
subs = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subs,str))

In [7]:
engines = Client()

In [16]:
def prep_fieldmaps(sub):        
    cal_dir = op.join(home_dir,'data',sub ,'cal')

    for sesh in ['a','b']:

        #first drop T1 saturation scans
        cals = glob.glob(cal_dir + '/cal_' + sesh + '*')
        sliced_scans = []
        for pe1 in cals:
            sliced = pe1[:-7] + '_slice'
            sliced_scans.append(sliced)
            cmd = ['fslroi',pe1,sliced,'3','2']
            cmd = ' '.join(cmd)
            os.system(cmd)

        #pe0 from main scans
        for run in range(1,7):
            out_pe0 = cal_dir + '/pe0_slice_' + sesh + str(run)
            in_f =  op.join(home_dir,'data',sub,'func',
                            'run_' + sesh + str(run) + '.nii.gz')
            data = nib.load(in_f)
            idx = data.shape[-1]//2 #middle volume
            cmd = ['fslroi',in_f,out_pe0,str(idx),'2']
            cmd = ' '.join(cmd)
            os.system(cmd) 

            #merge scans
            out_f = cal_dir + '/merged_' + sesh + str(run)
            if run <= 3: #get cal scan from that half of scanning
                idx = 0
            else:
                idx = 1
            cmd = ['fslmerge','-t',out_f,
                   sliced_scans[idx],
                  out_pe0]
            cmd = ' '.join(cmd)
            os.system(cmd)

            #motion correct
            cmd = ['mcflirt','-in',out_f,'-refvol','2'] #register to functional run
            os.system(' '.join(cmd))

In [9]:
sub_engines = engines[0:10] #less if you want to use fewer
sub_engines.push(dict(home_dir = home_dir))
sub_engines.execute('import os.path as op')
sub_engines.execute('import numpy as np')
sub_engines.execute('import nibabel as nib')
with sub_engines.sync_imports():
    import os
    import glob
    
output = sub_engines.map_sync(prep_fieldmaps,subs)

importing os on engine(s)
importing glob on engine(s)


CompositeError: one or more exceptions from call to method: prep_fieldmaps
[5:apply]: IndexError: list index out of range

In [18]:
##check cal
for sub in subs:
    cal_dir = op.join(home_dir,'data',sub ,'cal')
    for sesh in ['a','b']:
        for run in range(1,7):
            out_f = cal_dir + '/merged_' + sesh + str(run) + '_mcf.nii.gz'
            
            if not os.path.exists(out_f):
                print 'no file',sub,sesh,run

            elif nib.load(out_f).shape[-1] != 4:
                print 'wrong size',sub,sesh,run

    